In [1]:
from moku.instruments import MultiInstrument, LaserLockBox, DigitalFilterBox, FrequencyResponseAnalyzer, SpectrumAnalyzer
MIM = MultiInstrument('192.168.50.63', force_connect=True, platform_id=4) #192.168.50.63 is the IP address for Moku pro 4

# Set up multinstrument mode skeleton

In [2]:
llb = MIM.set_instrument(1, LaserLockBox)
fra = MIM.set_instrument(2, FrequencyResponseAnalyzer)
dfb = MIM.set_instrument(3, DigitalFilterBox)
spa = MIM.set_instrument(4, SpectrumAnalyzer)

In [3]:
connections = [dict(source="Input1", destination="Slot1InA"), # REFL PD to demodulator
               dict(source="Input2", destination="Slot1InB"), # TRANS PD to monitor
               dict(source="Slot1OutA", destination="Slot3InA"), # PID output to digital filter box
               dict(source="Slot1OutA", destination="Slot4InA"), 
               dict(source="Slot1OutA", destination="Slot2InB"),
               dict(source="Slot1OutC",destination="Output3"), # modulation to EOM
               dict(source="Slot2OutA", destination="Slot3InB"),
               dict(source="Slot3OutA", destination="Slot2InA"),
               dict(source="Slot3OutA", destination="Output1"), # control signal plus excitation goes to actuator
               dict(source="Slot3OutA",destination = "Slot4InA"),
               dict(source="Slot1OutA",destination = "Slot4InB")
               ]

MIM.set_connections(connections=connections)

[{'destination': 'Slot1InA', 'source': 'Input1'},
 {'destination': 'Slot1InB', 'source': 'Input2'},
 {'destination': 'Slot2InA', 'source': 'Slot3OutA'},
 {'destination': 'Slot2InB', 'source': 'Slot1OutA'},
 {'destination': 'Slot3InA', 'source': 'Slot1OutA'},
 {'destination': 'Slot3InB', 'source': 'Slot2OutA'},
 {'destination': 'Slot4InA', 'source': 'Slot3OutA'},
 {'destination': 'Slot4InB', 'source': 'Slot1OutA'},
 {'destination': 'Output1', 'source': 'Slot3OutA'},
 {'destination': 'Output3', 'source': 'Slot1OutC'}]

In [4]:
MIM.set_output(1, "0dB")
#MIM.set_output(2, "0dB") #to piezo
MIM.set_output(3, "14dB") #to EOM
MIM.set_frontend(1,"50Ohm","AC","0dB")
MIM.set_frontend(2,"1MOhm","DC","0dB")

{'attenuation': '0dB', 'coupling': 'DC', 'impedance': '1MOhm'}

# Set digital filter box to sum the excitation and control signal then apply an all-pass filter

In [5]:
dfb.set_control_matrix(1, input_gain1=1, input_gain2=1)
dfb.set_control_matrix(2, input_gain1=0, input_gain2=0)

filter_coefficients = [
    [1.0000000000, 1.0000000000, 0.0000000000, 0.0000000000, 0.0000000000, 0.0000000000],
    [1.0000000000, 1.0000000000, 0.0000000000, 0.0000000000, 0.0000000000, 0.0000000000],
    [1.0000000000, 1.0000000000, 0.0000000000, 0.0000000000, 0.0000000000, 0.0000000000],
    [1.0000000000, 1.0000000000, 0.0000000000, 0.0000000000, 0.0000000000, 0.0000000000]]

dfb.set_custom_filter(1, "39.06MHz", coefficients=filter_coefficients) 

dfb.set_input_gain(1,gain = 0)
dfb.set_output_gain(1,gain = 0)

dfb.enable_output(1,True,True)

{'output': True, 'signal': True}

# Configure the laser lock box

In [6]:
llb.set_aux_oscillator(source = "OutputC")
llb.set_demodulation(mode="Modulation",frequency=50e6,phase=260)
llb.set_output_limit(channel=1, low_limit=-1, high_limit=1)
llb.set_pid_by_frequency(channel=1,  int_crossover = 18e3, int_saturation = 40.0,prop_gain = 0)
print(llb.summary())

TypeError: LaserLockBox.set_aux_oscillator() got an unexpected keyword argument 'source'

In [ ]:
MIM.relinquish_ownership()